In [1]:
# get the code for kaggle
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = '/kaggle/working/Image-Captioning/'
import sys
sys.path.append(py_files_path)

fatal: destination path 'Image-Captioning' already exists and is not an empty directory.


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from nltk.translate.bleu_score import corpus_bleu

In [3]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 1  # number of epochs to train for (if early stopping is not triggered)
batch_size = 128
workers = 4
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder?
checkpoint = None  # path to checkpoint, None if none

In [4]:
DATA_NAME = 'flickr30k_5_cap_per_img_2_min_word_freq_resnet101'

# local
DATA_JSON_PATH = 'data30.json'
IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'

In [5]:
# load vocab
vocab = build_vocab(DATA_JSON_PATH)

100%|██████████| 145000/145000 [00:00<00:00, 297564.14it/s]


In [6]:
len(vocab)

11797

In [7]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

In [8]:
t_params

{'data_name': 'flickr30k_5_cap_per_img_2_min_word_freq_resnet101',
 'imgs_path': 'flickr/Images/',
 'df_path': 'data30.json',
 'vocab': <dataset.Vocabulary at 0x7fc8b5727050>,
 'epochs': 1,
 'batch_size': 128,
 'workers': 4,
 'decoder_lr': 0.0004,
 'encoder_lr': 0.0001,
 'fine_tune_encoder': False}

In [9]:
fit(t_params=t_params, m_params=m_params)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /home/kelwa/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

Loading Data
Dataset split: train
Unique images: 29000
Total size: 145000
Dataset split: val
Unique images: 1014
Total size: 5070
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/kelwa/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/kelwa/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/kelwa/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/media/kelwa/DEV/GP/Image-Captioning/dataset.py", line 52, in __getitem__
    img = Image.open(self.imgs_dir+img_id).convert("RGB")
  File "/home/kelwa/anaconda3/lib/python3.7/site-packages/PIL/Image.py", line 2912, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'flickr/Images/640290673.jpg'


<a href='BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101.pth.tar'>download</a>